In [52]:
from langchain.chains import create_sql_query_chain
from langchain_community.llms import Ollama

#llm = Ollama(model="tinyllama:latest",temperature=0)
from langchain_ollama import ChatOllama

llm = ChatOllama(
    model="llama3.1:latest",
    temperature=0)
#response = chain.invoke({"question": "How many customers are there"})
#response

In [53]:
from langchain_community.utilities import SQLDatabase

db = SQLDatabase.from_uri("sqlite:///Chinook.db")
print(db.dialect)
print(db.get_usable_table_names())
db.run("SELECT * FROM Artist LIMIT 10;")

sqlite
['Album', 'Artist', 'Customer', 'Employee', 'Genre', 'Invoice', 'InvoiceLine', 'MediaType', 'Playlist', 'PlaylistTrack', 'Track']


"[(1, 'AC/DC'), (2, 'Accept'), (3, 'Aerosmith'), (4, 'Alanis Morissette'), (5, 'Alice In Chains'), (6, 'Antônio Carlos Jobim'), (7, 'Apocalyptica'), (8, 'Audioslave'), (9, 'BackBeat'), (10, 'Billy Cobham')]"

In [54]:
from langchain_community.tools.sql_database.tool import QuerySQLDataBaseTool

execute_query = QuerySQLDataBaseTool(db=db)
write_query = create_sql_query_chain(llm, db)
chain = write_query | execute_query
#chain.invoke({"question": "How many customers are there"})

In [55]:
from langchain_core.runnables import RunnablePassthrough
from operator import itemgetter


In [56]:
from langchain_community.agent_toolkits import SQLDatabaseToolkit

toolkit = SQLDatabaseToolkit(db=db, llm=llm)

tools = toolkit.get_tools()

tools

[QuerySQLDataBaseTool(description="Input to this tool is a detailed and correct SQL query, output is a result from the database. If the query is not correct, an error message will be returned. If an error is returned, rewrite the query, check the query, and try again. If you encounter an issue with Unknown column 'xxxx' in 'field list', use sql_db_schema to query the correct table fields.", db=<langchain_community.utilities.sql_database.SQLDatabase object at 0x000001B6F6DBDA30>),
 InfoSQLDatabaseTool(description='Input to this tool is a comma-separated list of tables, output is the schema and sample rows for those tables. Be sure that the tables actually exist by calling sql_db_list_tables first! Example Input: table1, table2, table3', db=<langchain_community.utilities.sql_database.SQLDatabase object at 0x000001B6F6DBDA30>),
 ListSQLDatabaseTool(db=<langchain_community.utilities.sql_database.SQLDatabase object at 0x000001B6F6DBDA30>),
 QuerySQLCheckerTool(description='Use this tool to 

In [57]:
from langchain_core.messages import SystemMessage

SQL_PREFIX = """You are an agent designed to interact with a SQL database.
Given an input question, create a syntactically correct SQLite query to run, then look at the results of the query and return the answer.
Unless the user specifies a specific number of examples they wish to obtain, always limit your query to at most 5 results.
You can order the results by a relevant column to return the most interesting examples in the database.
Never query for all the columns from a specific table, only ask for the relevant columns given the question.
You have access to tools for interacting with the database.
Only use the below tools. Only use the information returned by the below tools to construct your final answer.
You MUST double check your query before executing it. If you get an error while executing a query, rewrite the query and try again.

DO NOT make any DML statements (INSERT, UPDATE, DELETE, DROP etc.) to the database.

To start you should ALWAYS look at the tables in the database to see what you can query.
Do NOT skip this step.
Then you should query the schema of the most relevant tables."""

system_message = SystemMessage(content=SQL_PREFIX)

In [58]:
from langchain_core.messages import HumanMessage
from langgraph.prebuilt import create_react_agent

agent_executor = create_react_agent(llm, tools, messages_modifier=system_message)


C:\Users\willi\AppData\Local\Temp\ipykernel_3892\1800557427.py:4: LangGraphDeprecationWarning: Parameter 'messages_modifier' in function 'create_react_agent' is deprecated as of version 0.1.9 and will be removed in version 0.3.0. Use 'state_modifier' parameter instead.
  agent_executor = create_react_agent(llm, tools, messages_modifier=system_message)


In [84]:
chain = (
    RunnablePassthrough.assign(query=write_query).assign(
        result=itemgetter("query") | execute_query
    )
    | answer_prompt
    | llm
    | StrOutputParser()
)

In [111]:
from langchain_community.tools.sql_database.tool import QuerySQLDataBaseTool

execute_query = QuerySQLDataBaseTool(db=db)
write_query = create_sql_query_chain(llm, db)
chain = RunnablePassthrough() | execute_query
#chain.invoke({"question": "How many employees are there"})

In [ ]:
from langchain_community.tools.sql_database.tool import QuerySQLDataBaseTool

execute_query = QuerySQLDataBaseTool(db=db)
write_query = create_sql_query_chain(llm, db)
chain = RunnablePassthrough() | execute_query
#chain.invoke({"question": "How many employees are there"})

In [186]:
from langchain_core.messages import HumanMessage
from langgraph.prebuilt import create_react_agent

agent_executor = create_react_agent(llm, tools, state_modifier=system_message)

In [193]:
testando_resultado = agent_executor.invoke({"messages":"How many model versions are there?"})

In [203]:
from operator import itemgetter

from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import PromptTemplate
from langchain_core.runnables import RunnablePassthrough

answer_prompt = PromptTemplate.from_template(
    """Given the following user question, corresponding SQL query, and SQL result, answer the user question.

Question: {question}
SQL Query: {query}
SQL Result: {result}
Answer: """
)

#lambda inputs: agent_executor.invoke(inputs)["messages"][-1].content

chain = (
     RunnablePassthrough().assign(query= lambda inputs: agent_executor.invoke(inputs)["messages"][-1].content).assign(result=itemgetter("query")
        | execute_query           
    )
    
    | answer_prompt

)

chain.invoke({"messages":"How many model versions are there?"})

StringPromptValue(text='Given the following user question, corresponding SQL query, and SQL result, answer the user question.\nHow many model versions are there?\nSQL Result: Error: (sqlite3.OperationalError) near "It": syntax error\n[SQL: It seems that the `model_versions` table does not exist in the database. Let me check again.\n\n{"name": "sql_db_tables", }]\n(Background on this error at: https://sqlalche.me/e/20/e3q8)\nAnswer: ')

In [127]:
for s in chain.stream(
"SELECT COUNT(*) FROM model_versions;"
):
    print(s)
    print("----")

TypeError: Expected mapping type as input to PromptTemplate. Received <class 'str'>.

In [112]:
chain.invoke("SELECT * FROM Artist LIMIT 10;")

"[(1, 'AC/DC'), (2, 'Accept'), (3, 'Aerosmith'), (4, 'Alanis Morissette'), (5, 'Alice In Chains'), (6, 'Antônio Carlos Jobim'), (7, 'Apocalyptica'), (8, 'Audioslave'), (9, 'BackBeat'), (10, 'Billy Cobham')]"

In [44]:
question = "How many model versions are there?"

In [96]:
final_output = chain.invoke({"SELECT * FROM Artist LIMIT 10;"})
print("Final Output:", final_output)

ValidationError: 1 validation error for _QuerySQLDataBaseToolInput
query
  Field required [type=missing, input_value={'question': {'SELECT * FROM Artist LIMIT 10;'}}, input_type=dict]
    For further information visit https://errors.pydantic.dev/2.9/v/missing

In [10]:
from langchain.agents import Tool

# Define a simple tool that converts Celsius to Fahrenheit
def celsius_to_fahrenheit(celsius):
    return (int(celsius) * 9/5) + 32

# Create the tool
temperature_tool = Tool(
    name="CelsiusToFahrenheit",
    func=celsius_to_fahrenheit,
    description="Converts Celsius to Fahrenheit."
)

# Invoke the tool
result = temperature_tool.run("25")  # Convert 25°C to Fahrenheit
print(result)


77.0


In [11]:
from langchain.agents import Tool

# Define a tool that fetches weather info (mocked example)
def get_weather(city):
    # Mocking an API call
    weather_data = {"New York": "Sunny", "London": "Rainy", "São Paulo": "Cloudy"}
    return "No data available"

# Create the tool
weather_tool = Tool(
    name="GetWeatherInfo",
    func=get_weather,
    description="Fetches current weather information for a city."
)

# Invoke the tool
result = weather_tool.run("São Paulo")
print(result)


No data available


In [15]:
from langchain.agents import AgentExecutor, Tool

# Define the tool for fetching weather
weather_tool = Tool(
    name="GetWeatherInfo",
    func=get_weather,
    description="Fetches weather information for a city."
)

# Create an agent with the weather tool
agent = AgentExecutor(tools=weather_tool)

# Ask the agent a question
response = agent.invoke({"input": "What's the weather in São Paulo?"})
print(response)


ValidationError: 2 validation errors for AgentExecutor
agent
  Field required [type=missing, input_value={'tools': Tool(name='GetW...at 0x000001B6F3C75120>)}, input_type=dict]
    For further information visit https://errors.pydantic.dev/2.9/v/missing
tools
  Input should be an instance of Sequence [type=is_instance_of, input_value=Tool(name='GetWeatherInfo... at 0x000001B6F3C75120>), input_type=Tool]
    For further information visit https://errors.pydantic.dev/2.9/v/is_instance_of

In [16]:
from langchain_core.runnables import Runnable

# Define a simple runnable
class SquareRunnable(Runnable):
    def invoke(self, input_data):
        return input_data ** 2

# Instantiate the runnable
square_runnable = SquareRunnable()

# Invoke it with an input
result = square_runnable.invoke(5)
print(result)  # Output will be 25


25
